# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122856e+02     1.634875e+00
 * time: 0.35370707511901855
     1     1.119221e+01     8.673287e-01
 * time: 2.16194486618042
     2    -1.200182e+01     1.033060e+00
 * time: 2.8036069869995117
     3    -3.408316e+01     7.871606e-01
 * time: 3.7528879642486572
     4    -4.770500e+01     5.806629e-01
 * time: 4.631706953048706
     5    -5.684546e+01     2.775951e-01
 * time: 5.534621953964233
     6    -5.972820e+01     1.918756e-01
 * time: 6.145789861679077
     7    -6.085722e+01     5.897904e-02
 * time: 6.749543905258179
     8    -6.129395e+01     8.250207e-02
 * time: 7.354156017303467
     9    -6.155021e+01     6.369585e-02
 * time: 7.963732957839966
    10    -6.175255e+01     2.751149e-02
 * time: 8.575260877609253
    11    -6.190148e+01     2.254178e-02
 * time: 9.178956031799316
    12    -6.197360e+01     2.118766e-02
 * time: 9.789720058441162
    13    -6.204873e+01     1.732777e-02
 * time: 10.387896060943604
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557651
    AtomicNonlocal      14.8522627
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485362 
    Xc                  -19.3336817

    total               -62.261666458230
